In [2]:
import xarray as xr
import math
from pyproj import CRS, Transformer
import pyproj
import numpy as np
import xesmf as xe
import matplotlib.pyplot as plt
import fsspec 
from dask.distributed import Client, progress
import re
from cfgrib import xarray_to_grib
import pickle
import gc
from datetime import date

Global Parameters

In [3]:
# Sets the threshold for storm strength. Anything greater than this value will be rejected.
CMP_MAX = 1000.0

# The AWS bucket the data is pulled from.
BUCKET_NAME = 'noaa-nws-hafs-pds'

# The variables removed from the hafs grib files. Remaining values are ['gh', 't', 'r', 'u', 'v']
DROP_VARIABLES = ['q', 'w', 'wz', 'absv', 'clwmr', 'icmr', 'rwmr', 'snmr','grle', 'rare', 'dpt']



In [4]:
# Setting up the dask client and Amazon s3 system
try:
    from dask.distributed import get_client
    get_client().close()
except Exception:
    pass

client = Client()  # set up local cluster on your laptop
client

fs = fsspec.filesystem('s3', anon=True)

Project Functions

In [6]:
def frame_processing(s3_path: str):
    '''
    Docstring for frame_processing
    
    :param ds: A HAFSA model timestep
    :type ds: xr.Dataset

    Checks for frame validity:
        No land within 200km of center
        Central minimum pressure of at least 990mb
    
    If valid returns a Model Dataset consisting of:
        PBL temp, heights, rh, and u/v winds
        700 mb temp, heights, rh, and u/v winds
        central minimum pressure for the frame,
        maximum wind for the frame
    '''
    
    header_path = 'simplecache::s3://noaa-nws-hafs-pds/'

    # A3 path for 24 hr timestep for a given storm and model run
    frame_path = header_path + s3_path

    # A3 path for 27 hr timestep for a given storm and model run. Necessary for determining storm motion
    nxt_frame_path = header_path = re.sub('.f024', '.f027', frame_path)

    # Loading the frames into cache
    frame_file = fsspec.open_local(frame_path, s3={'anon': True}, filecache={'cache_storage':'/tmp/files'})
    nxt_frame_file = fsspec.open_local(nxt_frame_path, s3={'anon': True}, filecache={'cache_storage':'/tmp/files'})

    # Loading the 24 surface frame. Primarily used for minimum central pressure.
    ds_sfc = xr.open_dataset(frame_file,  
                         filter_by_keys={'typeOfLevel': 'meanSea'},
                         engine = 'cfgrib')
    
    # Finding the storm center
    ds_sfc = ds_sfc['prmsl'].compute()/100
    c_mslp = ds_sfc.min()
    
    # Checking to see if the frame meets storm strength requirements
    # If not, returns None for the frame and moves onto the next frame.

    if c_mslp > CMP_MAX:
        print(f"{c_mslp.values} does not meet the threshold")
        return None, None
    else:
        
        center_coords = ds_sfc.where(ds_sfc == c_mslp, drop=True).squeeze()

    # Loading the Atmospheric Data
    ds_atm = xr.open_dataset(frame_file,
                         drop_variables = DROP_VARIABLES, 
                         filter_by_keys={'typeOfLevel': 'isobaricInhPa'},
                         engine = 'cfgrib'
                        )   
    
    # Loading the surface data for the next time step
    ds_sfc_nxt_step = xr.open_dataset(nxt_frame_file, 
                         filter_by_keys={'typeOfLevel': 'meanSea'},
                         engine = 'cfgrib')
    
    # Filtering by level to the 700mb and sfc level. Sfc level is determined by selecting
    # the level nearest to the minimum central pressure 
    ds_atm = ds_atm.sel(isobaricInhPa = [c_mslp.values, 700.0], method = 'nearest')
    
    
    # computing the necessary values for the follow on frame. Necessary
    # for determining storm motion

    ds_sfc_nxt_step = ds_sfc_nxt_step['prmsl'].compute()
    c_mslp_nxt_step = ds_sfc_nxt_step.min()
    center_coords_nxt_step = ds_sfc_nxt_step.where(ds_sfc_nxt_step == c_mslp_nxt_step, drop=True).squeeze()

    storm_heading = calc_heading(center_coords.latitude.values, center_coords.longitude.values, 
                                 center_coords_nxt_step.latitude.values, center_coords_nxt_step.longitude.values)
    
    max_wind = np.nanmax(np.sqrt(ds_atm['u'].isel(isobaricInhPa = 0).values**2 + ds_atm['v'].isel(isobaricInhPa = 0).values**2))
    
    # Rotating and regridding the dataset to polar coordinates
    frame = to_polar(ds_atm.compute(),
                     origin_lat= center_coords.latitude.values,
                     origin_lon= center_coords.longitude.values,
                     storm_heading = storm_heading)
    
    # Setting up the return objects
    atm_object = {
                    'radius_coords': frame.radius.values,
                    'angle_coords': frame.angle.values,
                    'gh': frame['gh'].isel(isobaricInhPa = 0).values,
                    't': frame['t'].isel(isobaricInhPa = 0).values,
                    'r': frame['r'].isel(isobaricInhPa = 0).values,
                    'u': frame['u'].isel(isobaricInhPa = 0).values,
                    'v': frame['v'].isel(isobaricInhPa = 0).values,
                    'center_lat': center_coords.latitude.values,
                    'center_lon': center_coords.longitude.values,
                    'valid_time': frame.isel(isobaricInhPa = 0).valid_time.values,
                    'storm_heading': storm_heading
                    }
    sfc_object = {
                    'radius_coords': frame.radius.values,
                    'angle_coords': frame.angle.values,
                    'gh': frame['gh'].isel(isobaricInhPa = 1).values,
                    't': frame['t'].isel(isobaricInhPa = 1).values,
                    'r': frame['r'].isel(isobaricInhPa = 1).values,
                    'u': frame['u'].isel(isobaricInhPa = 1).values,
                    'v': frame['v'].isel(isobaricInhPa = 1).values,
                    'center_pressure': c_mslp.values,
                    'max_wind': max_wind,
                    'center_lat': center_coords.latitude.values,
                    'center_lon': center_coords.longitude.values,
                    'valid_time': frame.valid_time.values,
                    'storm_heading': storm_heading
                    }
                    
    
    return sfc_object, atm_object
    
def calc_heading(lat1, lon1, lat2, lon2):
    '''
        Function to determine the storm heading. 
        Lat1 and Lon1 are the coordinates for the timestep.
        Lat2 and Lon2 are the coordinates for the follow-on timestep.

        returns a heading in degrees
    '''
    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    lon1_rad = np.deg2rad(lon1)
    lon2_rad = np.deg2rad(lon2)

    d_lon = lon2_rad - lon1_rad

    y = np.sin(d_lon) * np.cos(lat2_rad)
    x = (np.cos(lat1_rad) * np.sin(lat2_rad) -
         np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(d_lon)) 
    
    initial_bearing_rad = np.arctan2(y, x)

    initial_bearing_deg = np.rad2deg(initial_bearing_rad)

    bearing = (initial_bearing_deg + 360) % 360

    return bearing 

def rotate_point_around_origin(x, y, angle):
    """
    Rotates a 2D point (x,y) or array of points around the origin (0, 0) by a given angle.

    Args:
        x (float): The x-coordinate of the point or a meshgrid of x-coordinates.
        y (float): The y-coordinate of the point or a meshgrid of y-coordinates.
        angle_radians (float): The angle of rotation in radians (counter-clockwise).

    Returns:
        tuple: A tuple (new_x, new_y) representing the rotated point.
    """
    angle = -(np.deg2rad(angle))
    new_x = x * math.cos(angle) - y * math.sin(angle)
    new_y = x * math.sin(angle) + y * math.cos(angle)
    return new_x, new_y

def to_polar(
        ds: xr.Dataset,
        origin_lat: float,
        origin_lon: float,
        storm_heading: float,
        R_max = 250.0, # Max range in km
        R_step = 0.5, # Range step in km
        A_step = 0.5  # Azimuth step in degrees
    ) -> xr.Dataset:
    
        R = 6378137 # radius of Earth in meters

        new_range = np.arange(0.0, R_max + R_step, R_step)
        new_azimuth = np.arange(0.0, 360.0, A_step)
        R, T = np.meshgrid(new_azimuth, new_range, indexing = 'ij')
     
        X = R * np.cos(T) * 1000 # Convert km to meters
        Y = R * np.sin(T) * 1000
        local_proj_str = f"+proj=aeqd +lat_0={origin_lat} +lon_0={origin_lon} +units=m"
        transformer = pyproj.Transformer.from_crs(local_proj_str, "EPSG:4326", always_xy=True)

        X,Y = rotate_point_around_origin(X, Y, storm_heading)
        target_lons, target_lats = transformer.transform(X, Y)
        
        # print("Target Lats: " + str(np.shape(target_lats)))
        # print("Target Lons: " + str(np.shape(target_lons)))
        # print("range: " + str(np.shape(new_range)))
        # print("azimuth: " + str(np.shape(new_azimuth)))

        # print("New Range: " + str(new_range.max()))
        # print("New Angle: " + str(new_azimuth.max()))
        ds_out = xr.Dataset(
                            coords={
                                    "latitude": (("angle", "radius"), target_lats),
                                    "longitude": (("angle", "radius"), target_lons),
                                    "radius": new_range,
                                    "angle": new_azimuth
                                        }
                            )
        regridder = xe.Regridder(ds, ds_out, 'bilinear')
        polar_out = regridder(ds)
       
        return polar_out


         


In [ ]:
loaded_list = []

with open('Data/links/link_list.txt', 'r') as f:
            for line in f:
                loaded_list.append(line.strip())

In [8]:
starting_frame = 400
partial_loaded_list = loaded_list[starting_frame:(starting_frame+200)]

In [ ]:
frames_700mb = {
                    'radius_coords': [],
                    'angle_coords': [],
                    'gh': [],
                    't': [],
                    'r': [],
                    'u': [],
                    'v': [],
                    'valid_time': [],
                    'center_lat': [],
                    'center_lon': [],
                    'storm_heading': []
    }

frames_sfc = {
                    'radius_coords': [],
                    'angle_coords': [],
                    'gh': [],
                    't': [],
                    'r': [],
                    'u': [],
                    'v': [],
                    'center_pressure': [],
                    'max_wind': [],
                    'valid_time': [],
                    'center_lat': [],
                    'center_lon': [],
                    'storm_heading': []

    }

try:
    sfc_frame, atm_frame = frame_processing(partial_loaded_list[0])
    frame = True
    start = 0
    if not sfc_frame:
        frame = False

except Exception as e:
    print("Issue with link {loaded_list[0]}")
    print(e)
    frame = False

if frame:

    frames_700mb['radius_coords'].append(atm_frame['radius_coords'])
    frames_700mb['angle_coords'].append(atm_frame['angle_coords'])
    frames_700mb['gh'].append(atm_frame['gh'])
    frames_700mb['t'].append(atm_frame['t'])
    frames_700mb['r'].append(atm_frame['r'])
    frames_700mb['u'].append(atm_frame['u'])
    frames_700mb['v'].append(atm_frame['v'])
    frames_700mb['valid_time'].append(atm_frame['valid_time'])

    frames_sfc['radius_coords'].append(sfc_frame['radius_coords'])
    frames_sfc['angle_coords'].append(sfc_frame['angle_coords'])
    frames_sfc['gh'].append(sfc_frame['gh'])
    frames_sfc['t'].append(sfc_frame['t'])
    frames_sfc['r'].append(sfc_frame['r'])
    frames_sfc['u'].append(sfc_frame['u'])
    frames_sfc['v'].append(sfc_frame['v'])
    frames_sfc['center_pressure'].append(sfc_frame['center_pressure'])
    frames_sfc['max_wind'].append(sfc_frame['max_wind'])
    frames_sfc['valid_time'].append(sfc_frame['valid_time'])
    frames_sfc['center_lat'].append(sfc_frame['center_lat'])
    frames_sfc['center_lon'].append(sfc_frame['center_lon'])
    frames_sfc['storm_heading'].append(sfc_frame['storm_heading'])


for i, frame_path in enumerate(partial_loaded_list):
    
    print(i)
    try:
        sfc_frame, atm_frame = frame_processing(frame_path)
        frame = True
        if not sfc_frame:
            frame = False

    except Exception as e:
        print("Issue with link {frame_path}")
        print(e)
        frame = False
        
    if frame:
        
        frames_700mb['radius_coords'].append(atm_frame['radius_coords'])
        frames_700mb['angle_coords'].append(atm_frame['angle_coords'])
        frames_700mb['gh'].append(atm_frame['gh'])
        frames_700mb['t'].append(atm_frame['t'])
        frames_700mb['r'].append(atm_frame['r'])
        frames_700mb['u'].append(atm_frame['u'])
        frames_700mb['v'].append(atm_frame['v'])
        frames_700mb['valid_time'].append(atm_frame['valid_time'])
       

        frames_sfc['radius_coords'].append(sfc_frame['radius_coords'])
        frames_sfc['angle_coords'].append(sfc_frame['angle_coords'])
        frames_sfc['gh'].append(sfc_frame['gh'])
        frames_sfc['t'].append(sfc_frame['t'])
        frames_sfc['r'].append(sfc_frame['r'])
        frames_sfc['u'].append(sfc_frame['u'])
        frames_sfc['v'].append(sfc_frame['v'])
        frames_sfc['center_pressure'].append(sfc_frame['center_pressure'])
        frames_sfc['max_wind'].append(sfc_frame['max_wind'])
        frames_sfc['valid_time'].append(sfc_frame['valid_time'])
        frames_sfc['center_lat'].append(sfc_frame['center_lat'])
        frames_sfc['center_lon'].append(sfc_frame['center_lon'])
        frames_sfc['storm_heading'].append(sfc_frame['storm_heading'])
       
    # if i%10 == 0:
        # end = str(start + i)
        # with open("CompleteData/all_frames_" + str(start) + "to" + str(end) + ".pkl", "wb") as f:
        #     pickle.dump(all_frames, f)
        
ds = xr.Dataset(
                data_vars = {
                            'gh': (['levels','valid_time', 'angle', 'radius' ], 
                                np.stack([frames_sfc['gh'], frames_700mb['gh']]),
                                {'units': 'm', 'long_name': 'Geopotential Height'}),
                            'rh': (['levels','valid_time','angle', 'radius'], 
                                np.stack([frames_sfc['r'], frames_700mb['r']]),
                                {'units': 'percent humidity', 'long_name': 'Relative Humidity'}),
                            't': (['levels','valid_time', 'angle', 'radius'], 
                                np.stack([frames_sfc['t'], frames_700mb['t']]),
                                {'units': 'K', 'long_name': 'Air Temperature'}),
                            'u': (['levels','valid_time', 'angle', 'radius'], 
                                np.stack([frames_sfc['u'], frames_700mb['u']]),
                                {'units': 'm/s', 'long_name': 'U wind'}),
                            'v': (['levels','valid_time', 'angle', 'radius'], 
                                np.stack([frames_sfc['v'], frames_700mb['v']]),
                                {'units': 'm/s', 'long_name': 'V wind'}),
                            'center_pressure': (['valid_time'], 
                                                frames_sfc['center_pressure'],
                                                {'units': 'mb', 'long_name': 'Central Minimum Pressure in the Storm Environment'}),
                            'max_wind': (['valid_time'], 
                                        frames_sfc['max_wind'],
                                        {'units': 'm/s', 'long_name': 'Maximum Wind Speed in the Storm Environment'}),
                            'center_lat': (['valid_time'], 
                                        frames_sfc['center_lat'],
                                        {'units': 'degrees latitude', 'long_name': 'Latitude of Storm Center'}),
                            'center_lon': (['valid_time'], 
                                        frames_sfc['center_lon'],
                                        {'units': 'degrees longitude', 'long_name': 'Longitude of Storm Center'}),
                            'storm_heading': (['valid_time'], 
                                        frames_sfc['storm_heading'],
                                        {'units': 'degrees', 'long_name': 'Storm Heading relative to Lat/Lon Center'}),
                        },
                    coords= {
                            'angle': ('angle',
                                    frames_sfc['angle_coords'][0],
                                    {'units': 'degreees', 'long_name': 'Azimuth around Storm Center'}), 
                            'radius': ('radius',
                                    frames_sfc['radius_coords'][0],
                                    {'units': 'km', 'long_name': 'Range from Storm Center'}), 
                            'valid_time': frames_sfc['valid_time'],
                            'levels': (['sfc', 700])
                            },
                    attrs= {'history': f'Created on {date.today()}',
                            'Overview': ('Data originally obtained from the HAFS Repository ' 
                                            '(https://registry.opendata.aws/noaa-nws-hafs/) '
                                            'hosted on the AWS Sustainability Data Initiative. '
                                            'Data was regridded and interpolated from the original '
                                            'Lat/Lon grid to a regular polar grid centered on storm '
                                            'center. The center was determined by finding the lowest '
                                            'surface pressure within the surface pressure field. '
                                            'Storm heading was determined by finding the center location '
                                            'in the following time step and calculating the heading between '
                                            'those two points. Sfc field level data is determined by finding the '
                                            'pressure level in the dataset nearest the central minimum pressure.')}        
        )
ds.to_netcdf(f'CompleteData/StormData{start}To{start+200}.nc')
        
    #     start = i + start + 1
    #     frames_700mb = {
    #                     'radius_coords': [],
    #                     'angle_coords': [],
    #                     'gh': [],
    #                     't': [],
    #                     'r': [],
    #                     'u': [],
    #                     'v': [],
    #                     'valid_time': [],
    #                     'center_lat': [],
    #                     'center_lon': [],
    #                     'storm_heading': []
    #                     }

    #     frames_sfc = {
    #                     'radius_coords': [],
    #                     'angle_coords': [],
    #                     'gh': [],
    #                     't': [],
    #                     'r': [],
    #                     'u': [],
    #                     'v': [],
    #                     'center_pressure': [],
    #                     'max_wind': [],
    #                     'valid_time': [],
    #                     'center_lat': [],
    #                     'center_lon': [],
    #                     'storm_heading': []

    #                     }
    # del frame
    # gc.collect()    



Issue with link {loaded_list[0]}
name 'frame_processing' is not defined


NameError: name 'partial_loaded_list' is not defined

In [ ]:
starting_frame = 600
partial_loaded_list = loaded_list[starting_frame:(starting_frame+200)]

In [ ]:
frames_700mb = {
                    'radius_coords': [],
                    'angle_coords': [],
                    'gh': [],
                    't': [],
                    'r': [],
                    'u': [],
                    'v': [],
                    'valid_time': [],
                    'center_lat': [],
                    'center_lon': [],
                    'storm_heading': []
    }

frames_sfc = {
                    'radius_coords': [],
                    'angle_coords': [],
                    'gh': [],
                    't': [],
                    'r': [],
                    'u': [],
                    'v': [],
                    'center_pressure': [],
                    'max_wind': [],
                    'valid_time': [],
                    'center_lat': [],
                    'center_lon': [],
                    'storm_heading': []

    }

try:
    sfc_frame, atm_frame = frame_processing(loaded_list[0])
    frame = True
    start = 0
    if not sfc_frame:
        frame = False

except Exception as e:
    print("Issue with link {loaded_list[0]}")
    print(e)
    frame = False

if frame:

    frames_700mb['radius_coords'].append(atm_frame['radius_coords'])
    frames_700mb['angle_coords'].append(atm_frame['angle_coords'])
    frames_700mb['gh'].append(atm_frame['gh'])
    frames_700mb['t'].append(atm_frame['t'])
    frames_700mb['r'].append(atm_frame['r'])
    frames_700mb['u'].append(atm_frame['u'])
    frames_700mb['v'].append(atm_frame['v'])
    frames_700mb['valid_time'].append(atm_frame['valid_time'])

    frames_sfc['radius_coords'].append(sfc_frame['radius_coords'])
    frames_sfc['angle_coords'].append(sfc_frame['angle_coords'])
    frames_sfc['gh'].append(sfc_frame['gh'])
    frames_sfc['t'].append(sfc_frame['t'])
    frames_sfc['r'].append(sfc_frame['r'])
    frames_sfc['u'].append(sfc_frame['u'])
    frames_sfc['v'].append(sfc_frame['v'])
    frames_sfc['center_pressure'].append(sfc_frame['center_pressure'])
    frames_sfc['max_wind'].append(sfc_frame['max_wind'])
    frames_sfc['valid_time'].append(sfc_frame['valid_time'])
    frames_sfc['center_lat'].append(sfc_frame['center_lat'])
    frames_sfc['center_lon'].append(sfc_frame['center_lon'])
    frames_sfc['storm_heading'].append(sfc_frame['storm_heading'])


for i, frame_path in enumerate(loaded_list):
    
    print(i)
    try:
        sfc_frame, atm_frame = frame_processing(frame_path)
        frame = True
        if not sfc_frame:
            frame = False

    except Exception as e:
        print("Issue with link {frame_path}")
        print(e)
        frame = False
        
    if frame:
        
        frames_700mb['radius_coords'].append(atm_frame['radius_coords'])
        frames_700mb['angle_coords'].append(atm_frame['angle_coords'])
        frames_700mb['gh'].append(atm_frame['gh'])
        frames_700mb['t'].append(atm_frame['t'])
        frames_700mb['r'].append(atm_frame['r'])
        frames_700mb['u'].append(atm_frame['u'])
        frames_700mb['v'].append(atm_frame['v'])
        frames_700mb['valid_time'].append(atm_frame['valid_time'])
       

        frames_sfc['radius_coords'].append(sfc_frame['radius_coords'])
        frames_sfc['angle_coords'].append(sfc_frame['angle_coords'])
        frames_sfc['gh'].append(sfc_frame['gh'])
        frames_sfc['t'].append(sfc_frame['t'])
        frames_sfc['r'].append(sfc_frame['r'])
        frames_sfc['u'].append(sfc_frame['u'])
        frames_sfc['v'].append(sfc_frame['v'])
        frames_sfc['center_pressure'].append(sfc_frame['center_pressure'])
        frames_sfc['max_wind'].append(sfc_frame['max_wind'])
        frames_sfc['valid_time'].append(sfc_frame['valid_time'])
        frames_sfc['center_lat'].append(sfc_frame['center_lat'])
        frames_sfc['center_lon'].append(sfc_frame['center_lon'])
        frames_sfc['storm_heading'].append(sfc_frame['storm_heading'])
       
    # if i%10 == 0:
        # end = str(start + i)
        # with open("CompleteData/all_frames_" + str(start) + "to" + str(end) + ".pkl", "wb") as f:
        #     pickle.dump(all_frames, f)
        
ds = xr.Dataset(
                data_vars = {
                            'gh': (['levels','valid_time', 'angle', 'radius' ], 
                                np.stack([frames_sfc['gh'], frames_700mb['gh']]),
                                {'units': 'm', 'long_name': 'Geopotential Height'}),
                            'rh': (['levels','valid_time','angle', 'radius'], 
                                np.stack([frames_sfc['r'], frames_700mb['r']]),
                                {'units': 'percent humidity', 'long_name': 'Relative Humidity'}),
                            't': (['levels','valid_time', 'angle', 'radius'], 
                                np.stack([frames_sfc['t'], frames_700mb['t']]),
                                {'units': 'K', 'long_name': 'Air Temperature'}),
                            'u': (['levels','valid_time', 'angle', 'radius'], 
                                np.stack([frames_sfc['u'], frames_700mb['u']]),
                                {'units': 'm/s', 'long_name': 'U wind'}),
                            'v': (['levels','valid_time', 'angle', 'radius'], 
                                np.stack([frames_sfc['v'], frames_700mb['v']]),
                                {'units': 'm/s', 'long_name': 'V wind'}),
                            'center_pressure': (['valid_time'], 
                                                frames_sfc['center_pressure'],
                                                {'units': 'mb', 'long_name': 'Central Minimum Pressure in the Storm Environment'}),
                            'max_wind': (['valid_time'], 
                                        frames_sfc['max_wind'],
                                        {'units': 'm/s', 'long_name': 'Maximum Wind Speed in the Storm Environment'}),
                            'center_lat': (['valid_time'], 
                                        frames_sfc['center_lat'],
                                        {'units': 'degrees latitude', 'long_name': 'Latitude of Storm Center'}),
                            'center_lon': (['valid_time'], 
                                        frames_sfc['center_lon'],
                                        {'units': 'degrees longitude', 'long_name': 'Longitude of Storm Center'}),
                            'storm_heading': (['valid_time'], 
                                        frames_sfc['storm_heading'],
                                        {'units': 'degrees', 'long_name': 'Storm Heading relative to Lat/Lon Center'}),
                        },
                    coords= {
                            'angle': ('angle',
                                    frames_sfc['angle_coords'][0],
                                    {'units': 'degreees', 'long_name': 'Azimuth around Storm Center'}), 
                            'radius': ('radius',
                                    frames_sfc['radius_coords'][0],
                                    {'units': 'km', 'long_name': 'Range from Storm Center'}), 
                            'valid_time': frames_sfc['valid_time'],
                            'levels': (['sfc', 700])
                            },
                    attrs= {'history': f'Created on {date.today()}',
                            'Overview': ('Data originally obtained from the HAFS Repository ' 
                                            '(https://registry.opendata.aws/noaa-nws-hafs/) '
                                            'hosted on the AWS Sustainability Data Initiative. '
                                            'Data was regridded and interpolated from the original '
                                            'Lat/Lon grid to a regular polar grid centered on storm '
                                            'center. The center was determined by finding the lowest '
                                            'surface pressure within the surface pressure field. '
                                            'Storm heading was determined by finding the center location '
                                            'in the following time step and calculating the heading between '
                                            'those two points. Sfc field level data is determined by finding the '
                                            'pressure level in the dataset nearest the central minimum pressure.')}        
        )
ds.to_netcdf(f'CompleteData/StormData{start}To{start+200}.nc')